### Monday, January 29, 2024

Trying this again ... but with the conda environment 'langchain2'

1) mamba install conda-forge::numexpr ... NOPE! Tons of conflicts!
1) pip install numexpr ... no warnings, it just downloads numexpr version 2.9.0 

This all runs. 

### Friday, January 26, 2024

Trying this notebook using the LMStudio server from the 'langchain' conda environment.

Nice! This all runs in one pass!

### Monday, November 27, 2023

Re-running this notebook ...

Start : $1.65

### Monday, November 20, 2023

[LLM Chains using GPT 3.5 and other LLMs — LangChain #3](https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb)


In [1]:
# !pip install -qU langchain openai   

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/02-langchain-chains.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Getting Started with Chains

Chains are the core of LangChain. They are simply a chain of components, executed in a particular order.

The simplest of these chains is the `LLMChain`. It works by taking a user's input, passing in to the first element in the chain — a `PromptTemplate` — to format the input into a particular prompt. The formatted prompt is then passed to the next (and final) element in the chain — a LLM.

We'll start by importing all the libraries that we'll be using in this example.

In [1]:
import inspect
import re

from getpass import getpass
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain.callbacks import get_openai_callback

To run this notebook, we will need to use an OpenAI LLM. Here we will setup the LLM we will use for the whole notebook, just input your openai api key when prompted. 

In [2]:
# Yeah, this should be done in any notebook that uses OpenAI
# OPENAI_API_KEY = getpass()

In [3]:
import os
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [4]:
# llm = OpenAI(
#     temperature=0, 
#     openai_api_key=OPENAI_API_KEY
#     )

In [5]:
llm = OpenAI(base_url="http://localhost:1234/v1", api_key="NULL")

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
# what is the default temperature?
llm.temperature

0.7

In [8]:
# set it to what is expected for this example.
llm.temperature = 0.0

An extra utility we will use is this function that will tell us how many tokens we are using in each call. This is a good practice that is increasingly important as we use more complex tools that might make several calls to the API (like agents). It is very important to have a close control of how many tokens we are spending to avoid unsuspected expenditures.

In [9]:
def count_tokens(chain, query):
    
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

## What are chains anyway?

**Definition**: Chains are one of the fundamental building blocks of this lib (as you can guess!).

The official definition of chains is the following:


> A chain is made up of links, which can be either primitives or other chains. Primitives can be either prompts, llms, utils, or other chains.


So a chain is basically a pipeline that processes an input by using a specific combination of primitives. Intuitively, it can be thought of as a 'step' that performs a certain set of operations on an input and returns the result. They can be anything from a prompt-based pass through a LLM to applying a Python function to a text.

Chains are divided in three types: Utility chains, Generic chains and Combine Documents chains. In this edition, we will focus on the first two since the third is too specific (will be covered in due course).

1. Utility Chains: chains that are usually used to extract a specific answer from a llm with a very narrow purpose and are ready to be used out of the box.
2. Generic Chains: chains that are used as building blocks for other chains but cannot be used out of the box on their own.

Let's take a peek into what these chains have to offer!

### Utility Chains

Let's start with a simple utility chain. The `LLMMathChain` gives llms the ability to do math. Let's see how it works!

#### Pro-tip: use `verbose=True` to see what the different steps in the chain are!

#### numexpr side note

(The first time I run this, I get an error about the missing library 'numexpr', which of course means the LangChain LLMMathChain utility chain uses this library. However, when I then ran 'mamba install conda-forge::numexpr, I got back a ton of library conflicts, and so nothing was installed. So maybe an earlier release will work?? Looking at the [git repo](https://github.com/pydata/numexpr) it looks like version 2.8.7 from Sept 26, 2023 is worth trying so I ran 'mamba install conda-forge::numexpr==2.8.7' but NOPE! still a ton of conflicts!... Hmm so is there a way I can find out which version of this library should work?? Wait ... do I need to use = instead of == ? ... hmm nope same conflicts. Fuck it ... gonna run 'pip install numexpr' )

NumExpr is a fast numerical expression evaluator for NumPy. With it, expressions that operate on arrays (like '3*a+4*b') are accelerated and use less memory than doing the same calculation in Python.

I grabbed the following numexpr code from their [github repo](https://github.com/pydata/numexpr)

In [14]:
import numpy as np
import numexpr as ne

In [16]:
a = np.arange(1e6)
b = np.arange(1e6)

In [17]:
ne.evaluate("a + 1")  # a simple expression

array([1.00000e+00, 2.00000e+00, 3.00000e+00, ..., 9.99998e+05,
       9.99999e+05, 1.00000e+06])

In [18]:
ne.evaluate('a*b-4.1*a > 2.5*b') # a more complex expression

array([False, False, False, ...,  True,  True,  True])

In [19]:
ne.evaluate("sin(a) + arcsinh(a/b)")   # you can also use functions

array([        nan,  1.72284457,  1.79067101, ...,  1.09567006,
        0.17523598, -0.09597844])

In [20]:
s = np.array([b'abba', b'abbb', b'abbcdef'])

In [21]:
ne.evaluate("b'abba' == s")   # string arrays are supported too

array([ True, False, False])

 Notice how we express the expression as a string ... according to the [numexpr docs](https://numexpr.readthedocs.io/en/latest/index.html)

In [49]:
numericalExpression = "13**0.3432"

In [50]:
ne.evaluate(numericalExpression)

array(2.41160046)

So very interesting! The next cell works if we use 'mistral instruct v0.2', but fails with 'nexusraven v2', which is a bigger model. Bigger is not always better, right?!

It also fails with 'phi2'

And this is why I love LMStudio. I can switch out models on the fly!

In [62]:
llm_math = LLMMathChain(llm=llm, verbose=True)
# here we express the expression in more natural language ...
count_tokens(llm_math, "What is 13 raised to the .3432 power?")

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13**0.3432
```
...numexpr.evaluate("13**0.3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 70 tokens


'Answer: 2.4116004626599237'

Let's see what is going on here. The chain recieved a question in natural language and sent it to the llm. The llm returned a Python code which the chain compiled to give us an answer. A few questions arise.. How did the llm know that we wanted it to return Python code? 

**Enter prompts**

The question we send as input to the chain is not the only input that the llm recieves 😉. The input is inserted into a wider context, which gives precise instructions on how to interpret the input we send. This is called a _prompt_. Let's see what this chain's prompt is!

In [24]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



Ok.. let's see what we got here. So, we are literally telling the llm that for complex math problems **it should not try to do math on its own** but rather it should print a Python code that will calculate the math problem instead. Probably, if we just sent the query without any context, the llm would try (and fail) to calculate this on its own. Wait! This is testable.. let's try it out! 🧐

In [69]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# we ask the llm for the answer with no context

count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

Spent a total of 408 tokens


'\nThe answer is approximately 2.0857.\n\nTo calculate this value, you can use the following formula:\n\n13^(1/3.432) = 2.085695...\n\nSo, 13 raised to the power of .3432 is approximately equal to 2.0857.'

Wrong answer! Herein lies the power of prompting and one of our most important insights so far: 

**Insight**: _by using prompts intelligently, we can force the llm to avoid common pitfalls by explicitly and purposefully programming it to behave in a certain way._

Another interesting point about this chain is that it not only runs an input through the llm but it later compiles Python code. Let's see exactly how this works.

In [70]:
# The inspect module provides several useful functions to help get information about live objects 
# such as modules, classes, methods, functions, tracebacks, frame objects, and code objects. 
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



So we can see here that if the llm returns Python code we will compile it with a Python REPL* simulator (Read-Eval-Print Loop). We now have the full picture of the chain: either the llm returns an answer (for simple math problems) or it returns Python code which we compile for an exact answer to harder problems. Smart!

Also notice that here we get our first example of **chain composition**, a key concept behind what makes langchain special. We are using the `LLMMathChain` which in turn initializes and uses an `LLMChain` (a 'Generic Chain') when called. We can make any arbitrary number of such compositions, effectively 'chaining' many such chains to achieve highly complex and customizable behaviour.

Utility chains usually follow this same basic structure: there is a prompt for constraining the llm to return a very specific type of response from a given query. We can ask the llm to create SQL queries, API calls and even create Bash commands on the fly 🔥

The list continues to grow as langchain becomes more and more flexible and powerful so we encourage you to [check it out](https://langchain.readthedocs.io/en/latest/modules/chains/utility_how_to.html) and tinker with the example notebooks that you might find interesting.

*_A Python REPL (Read-Eval-Print Loop) is an interactive shell for executing Python code line by line_

### Generic chains

There are only three Generic Chains in langchain and we will go all in to showcase them all in the same example. Let's go!

Say we have had experience of getting dirty input texts. Specifically, as we know, llms charge us by the number of tokens we use and we are not happy to pay extra when the input has extra characters. Plus its not neat 😉

First, we will build a custom transform function to clean the spacing of our texts. We will then use this function to build a chain where we input our text and we expect a clean text as output.

In [71]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

Importantly, when we initialize the chain we do not send an llm as an argument. As you can imagine, not having an llm makes this chain's abilities much weaker than the example we saw earlier. However, as we will see next, combining this chain with other chains can give us highly desirable results.

In [72]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [73]:
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

Great! Now things will get interesting.

Say we want to use our chain to clean an input text and then paraphrase the input in a specific style, say a poet or a policeman. As we now know, the `TransformChain` does not use a llm so the styling will have to be done elsewhere. That's where our `LLMChain` comes in. We know about this chain already and we know that we can do cool things with smart prompting so let's take a chance!

First we will build the prompt template:

In [74]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

And next, initialize our chain:

In [75]:
style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

Great! Notice that the input text in the template is called 'output_text'. Can you guess why?

We are going to pass the output of the `TransformChain` to the `LLMChain`!

Finally, we need to combine them both to work as one integrated chain. For that we will use `SequentialChain` which is our third generic chain building block.

In [76]:
sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain], input_variables=['text', 'style'], output_variables=['final_output'])

Our input is the langchain docs description of what chains are but dirty with some extra spaces all around.

In [77]:
input_text = """
Chains allow us to combine multiple 


components together to create a single, coherent application. 

For example, we can create a chain that takes user input,       format it with a PromptTemplate, 

and then passes the formatted response to an LLM. We can build more complex chains by combining     multiple chains together, or by 


combining chains with other components.
"""

We are all set. Time to get creative!

In [78]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

Spent a total of 607 tokens


"Yo, check it out, we got these chains, man,\nLinkin' up components like a master plan,\nTake user input, put it through a template,\nPromptTemplate makin' it look so neat and simple,\nThen pass that formatted response to an LLM,\nLet the language learnin' magic begin, it's a sight to see,\nBut we ain't stoppin' there, we buildin' up the complexity,\nLinkin' chains together, makin' 'em all intertwine,\nOr mix and match with other components,\nCreatin' applications that'll make your heart sing."

(I did not run the next few cells because I don't want to use OpenAI)

## A note on langchain-hub

`langchain-hub` is a sister library to `langchain`, where all the chains, agents and prompts are serialized for us to use.

In [82]:
from langchain.chains import load_chain

In [81]:
# This needs to run for the load_chain command in the next cell to work ...
# actually I think it doesn't matter ...
import os
os.environ['OPENAI_API_KEY'] = 'sk-asdf' # replace any existing key with this bogus key ...

Loading from langchain hub is as easy as finding the chain you want to load in the repository and then using `load_chain` with the corresponding path. We also have `load_prompt` and `initialize_agent`, but more on that later. Let's see how we can do this with our `LLMMathChain` we saw earlier:

In [83]:
llm_math_chain = load_chain('lc://chains/llm-math/chain.json')

/home/rob/miniforge3/envs/langchain2/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [87]:
# Let's inspect this object ... it should be the same as above.
print(inspect.getsource(llm_math_chain._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



Go here [LangChain Hub](https://smith.langchain.com/hub) for a list of all available chains.

What if we want to change some of the configuration parameters? We can simply override it after loading:

In [84]:
llm_math_chain.verbose

True

In [85]:
llm_math_chain.verbose = False

In [86]:
llm_math_chain.verbose

False

That's it for this example on chains.

---